# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Connect to Snowflake

In [4]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [155]:
# Create some test data to work with
salary_list = [
    ["Alice Johnson", 10],
    ["Bob Smith", 20],
    ["Carlos Ray", 30],
    ["Diana Grace", 40],
    ["Evan Zhao", 50]
]

df = session.create_dataframe(salary_list, schema=['EMPLOYEE','SALARY'])
df.show(n=15, max_width=1000)

----------------------------
|"EMPLOYEE"     |"SALARY"  |
----------------------------
|Alice Johnson  |10        |
|Bob Smith      |20        |
|Carlos Ray     |30        |
|Diana Grace    |40        |
|Evan Zhao      |50        |
----------------------------



# Register & Run Table-QA Model

In [8]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [9]:
# Get the table-qa model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from transformers import pipeline
pipe = pipeline(task="table-question-answering", model="google/tapas-large-finetuned-wtq")

# use Snowpark Optimized Warehouse because model is big
session.use_warehouse('snowpark_opt_wh')

# Register the model to Snowflake (predict is the model's function we want to call)
snow_model = reg.log_model(
    pipe, 
    model_name='tapas_large_finetuned_wtq', 
    sample_input_data=df.limit(10),
    conda_dependencies=['tokenizers','transformers','sentencepiece']
    )

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'temperature': 0.0352513}


In [156]:
# transform data suitable for table-qa model
df = df.select(F.array_agg('EMPLOYEE').as_('EMPLOYEE'),F.array_agg(F.col('SALARY').cast(T.StringType())).as_('SALARY'))
df = df.select(F.call_builtin('object_agg', F.lit('EMPLOYEE'), F.col('EMPLOYEE')).as_('EMPLOYEE'), F.call_builtin('object_agg', F.lit('SALARY'), F.col('SALARY')).as_('SALARY'))
df = df.select(F.object_construct('*').cast(T.StringType()).as_('"table"'))
df.show()

------------------------------------------------------
|"table"                                             |
------------------------------------------------------
|{"EMPLOYEE":["Alice Johnson","Bob Smith","Carlo...  |
------------------------------------------------------



In [157]:
# Ask questions
df_query = df.with_column('"query"', F.lit('What is the salary of Diana Grace?').cast(T.StringType()))
snow_model.run(df_query).show(max_width=1000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"table"                                                                                                                |"query"                             |"answer"  |"coordinates"  |"cells"  |"aggregator"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|{"EMPLOYEE":["Alice Johnson","Bob Smith","Evan Zhao","Diana Grace","Carlos Ray"],"SALARY":["10","20","50","40","30"]}  |What is the salary of Diana Grace?  |SUM > 40  |[              |[        |SUM           |
|                                                                                                                       |                                   

In [159]:
# Ask questions
df_query = df.with_column('"query"', F.lit('What is the minimum salary?').cast(T.StringType()))
snow_model.run(df_query).show(max_width=1000)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"table"                                                                                                                |"query"                      |"answer"  |"coordinates"  |"cells"  |"aggregator"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|{"EMPLOYEE":["Diana Grace","Alice Johnson","Bob Smith","Carlos Ray","Evan Zhao"],"SALARY":["40","10","20","30","50"]}  |What is the minimum salary?  |SUM > 10  |[              |[        |SUM           |
|                                                                                                                       |                             |          |  [            |  "10"

In [160]:
# Ask questions
df_query = df.with_column('"query"', F.lit('Who has the highest salary?').cast(T.StringType()))
snow_model.run(df_query).show(max_width=1000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"table"                                                                                                                |"query"                      |"answer"   |"coordinates"  |"cells"        |"aggregator"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|{"EMPLOYEE":["Alice Johnson","Carlos Ray","Evan Zhao","Bob Smith","Diana Grace"],"SALARY":["10","30","50","20","40"]}  |Who has the highest salary?  |Evan Zhao  |[              |[              |NONE          |
|                                                                                                                       |                             |     

In [161]:
# Ask questions
df_query = df.with_column('"query"', F.lit('Which employees have a higher salary than Carlos Ray?').cast(T.StringType()))
snow_model.run(df_query).show(max_width=1000)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"table"                                                                                                                |"query"                                                |"answer"                |"coordinates"  |"cells"          |"aggregator"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|{"EMPLOYEE":["Bob Smith","Alice Johnson","Evan Zhao","Diana Grace","Carlos Ray"],"SALARY":["20","10","50","40","30"]}  |Which employees have a higher salary than Carlos Ray?  |Evan Zhao, Diana Grace  |[              |[                |NONE    